# Library imports, configurations

In [15]:
import os
import re
import s3fs
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from unidecode import unidecode
from wordcloud import WordCloud
import nltk
nltk.download('punkt')

from stop_words import get_stop_words
import unicodedata

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [17]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep',
 'jplaton/diffusion/ted',
 'jplaton/diffusion/visio_mel']

In [4]:
BUCKET_CLEM = "clementg/diffusion"
fs.ls(BUCKET_CLEM)

['clementg/diffusion/.keep',
 'clementg/diffusion/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv']

# Functions

In [18]:
# Suppression des accents

def remove_accent(s):

    s = unidecode(str(s))
    return(s)

# Suppression de la ponctuation

def remove_punctuation(s):
    # On retire la ponctuation
 
    s = re.sub(r'[^A-Za-z]',' ',str(s))
    return(s)

# Réduction de la casse

def tolower(s):
    s = ' '.join(str(s).lower().split())
    return(s)


# Sur cette base, on crée une petite fonction qui retire les stop words
stopwords = get_stop_words('french')   
stopwords = [unicodedata.normalize('NFKD',m).encode('ASCII', 'ignore').decode() for m in stopwords]

def remove_stopwords(s):
    s = [w for w in word_tokenize(s) if w not in stopwords]
    s = ' '.join(s)
    return(s)

def remove_accent_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
    return df_c

def remove_punctuation_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
    return df_c

def tolower_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(tolower)
    return df_c

# DATA Download

In [19]:
FILE_PATH_TED = "ted"
FILE_PATH_TED_S3 = BUCKET + "/" + FILE_PATH_TED

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20230409.csv", mode="rb") as file_in:
    offres_2023_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20221225.csv", mode="rb") as file_in:   
    offres_2022_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/Bulloterie_10_Saison_2.xlsx", mode="rb") as file_in:  
    bulloterie_df =  pd.read_excel(file_in, sheet_name=0,header=[0,1])


/tmp/ipykernel_7581/419981861.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2023_df = pd.read_csv(file_in, sep=";")
/tmp/ipykernel_7581/419981861.py:8: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2022_df = pd.read_csv(file_in, sep=";")


In [ ]:
with fs.open(BUCKET_CLEM+"/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv", mode="rb") as file_in:
    fdp_dinum_df = pd.read_csv(file_in, sep=";",encoding='Windows-1252')

In [21]:
with fs.open(FILE_PATH_TED_S3+"/dila_refOrga_admin_Etat_fr_20230505.json", mode="rb") as file_in:  
    orga =  pd.read_json(file_in)

In [57]:
orga

,service
0,"{'plage_ouverture': [], 'site_internet': [{'libelle': '', 'valeur': 'http://www.police-nationale.interieur.gouv.fr'}], 'copyright': 'Direction de l'information légale et administrative (Première ministre)', 'siren': '', 'ancien_code_pivot': '', 'texte_reference': [], 'partenaire': '', 'telecopie': ['01 47 44 97 99'], 'nom': 'Bureau de coordination stratégique', 'siret': '', 'itm_identifiant': '184318', 'sigle': '', 'date_modification': '04/02/2021 18:26:23', 'adresse_courriel': [], 'service_disponible': '', 'organigramme': [], 'pivot': [], 'partenaire_identifiant': '', 'ancien_identifiant': [], 'id': '00014eb0-f7fe-4988-9380-137ccdfc401e', 'ancien_nom': [], 'commentaire_plage_ouverture': '', 'annuaire': [], 'hierarchie': [], 'categorie': 'SI', 'sve': ['https://contacts-demarches.interieur.gouv.fr'], 'telephone_accessible': [], 'version_type': 'Publiable', 'type_repertoire': '', 'telephone': [{'valeur': '01 49 27 49 27', 'description': ''}], 'version_etat_modification': '', 'date_creation': '10/08/2009 00:00:00', 'partenaire_date_modification': '', 'mission': '', 'formulaire_contact': ['http://www.police-nationale.interieur.gouv.fr/Menu-accessibilite/Nous-contacter'], 'version_source': '', 'type_organisme': 'Administration centrale (ou Ministère)', 'code_insee_commune': '', 'statut_de_diffusion': 'true', 'adresse': [{'type_adresse': 'Adresse', 'complement1': '', 'complement2': '', 'numero_voie': '11 rue des Saussaies', 'service_distribution': '', 'code_postal': '75008', 'nom_commune': 'Paris', 'pays': '', 'continent': '', 'longitude': '', 'latitude': '', 'accessibilite': '', 'note_accessibilite': ''}], 'information_complementaire': '', 'date_diffusion': ''}"
1,"{'plage_ouverture': [], 'site_internet': [{'libelle': '', 'valeur': 'https://www.dila.premier-ministre.gouv.fr'}], 'copyright': 'Direction de l'information légale et administrative (Première ministre)', 'siren': '130009186', 'ancien_code_pivot': '', 'texte_reference': [{'libelle': 'Missions de la DILA (décret)', 'valeur': 'http://www.legifrance.gouv.fr/affichTexte.do?cidTexte=JORFTEXT000021658499'}, {'libelle': 'Organisation de la DILA (arrêté)', 'valeur': 'https://www.legifrance.gouv.fr/affichTexte.do;?cidTexte=JORFTEXT000033781192'}], 'partenaire': '', 'telecopie': [], 'nom': 'Direction de l'information légale et administrative', 'siret': '13000918600011', 'itm_identifiant': '185945', 'sigle': 'DILA', 'date_modification': '17/03/2023 14:31:25', 'adresse_courriel': [], 'service_disponible': '', 'organigramme': [], 'pivot': [], 'partenaire_identifiant': '', 'ancien_identifiant': [], 'id': '0001945a-3c57-4271-b3cd-38ce8ff2c696', 'ancien_nom': [], 'commentaire_plage_ouverture': '', 'annuaire': [], 'hierarchie': [{'type_hierarchie': 'Service Fils', 'service': '68c54a12-2377-4d44-b170-c210312d37b6'}, {'type_hierarchie': 'Service Fils', 'service': '53735d1c-3257-47ad-9197-63fd7edfac36'}, {'type_hierarchie': 'Service Fils', 'service': 'd77756bc-6c8d-4a4f-8a1d-5fefe11a6729'}, {'type_hierarchie': 'Service Fils', 'service': '8df3ee3a-3db4-4bda-9780-89d958c74321'}], 'categorie': 'SI', 'sve': ['https://www.dila.premier-ministre.gouv.fr/contacts/saisine-sve'], 'telephone_accessible': [{'valeur': 'https://www.acce-o.fr/client/dila', 'description': 'Du lundi au vendredi de 9h à 17h30', 'custom_dico1': ['Transcription automatique']}], 'version_type': 'Publiable', 'type_repertoire': '', 'telephone': [{'valeur': '01 40 58 75 00', 'description': ''}], 'version_etat_modification': '', 'date_creation': '06/01/2010 00:00:00', 'partenaire_date_modification': '', 'mission': 'La direction de l’information légale et administrative (DILA) est une direction d’administration centrale des services du Premier ministre, placée sous l’autorité du Secrétaire général du Gouvernement. Créée par décret le 11 janvier 2010, elle résulte de la fusion des directions de La Documentation française et des Journaux officiels. La DILA exerce les missions de diffusion légale, d’édition publique et d’info

In [44]:
json_column=orga['service']

In [64]:
frame = {'service_name': json_column.apply(lambda x: x['nom']),
'type_organisme':}

In [65]:
service_name = pd.DataFrame(frame)

In [66]:
service_name

,service_name
0,Bureau de coordination stratégique
1,Direction de l'information légale et administrative
2,Bureau de la simplification et de la qualité de service
3,Service de la protection (SDLP)
4,Service national d'accueil téléphonique de l'enfance en danger (SNATED)
...,...
7631,Observatoire de la santé des vétérans
7632,"Bureau du pilotage, de l'organisation et des moyens"
7633,"Bureau des personnels administratifs, techniques et scientifiques de la police nationale (BPATS)"
7634,Établissement français du sang (EFS) - Auvergne-Rhône-Alpes


In [53]:
service_name_clean = ( service_name
    .pipe(remove_accent_from_df,['service'])
    .pipe(remove_punctuation_from_df,['service'])
    .pipe(tolower_df,['service'])
)

/tmp/ipykernel_7581/701967925.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
/tmp/ipykernel_7581/701967925.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
/tmp/ipykernel_7581/701967925.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(tolower)


In [54]:
services_list = service_name_clean['service'].tolist()

In [11]:
offres_df = pd.concat([offres_2023_df,offres_2022_df])

In [24]:
offres_df.shape

(406755, 30)

In [13]:
offres_df_clean = ( offres_df
    .pipe(remove_accent_from_df,['Métier','Intitulé du poste'])
    .pipe(remove_punctuation_from_df,['Métier','Intitulé du poste'])
    .pipe(tolower_df,['Métier','Intitulé du poste'])
)

/tmp/ipykernel_7581/701967925.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
/tmp/ipykernel_7581/701967925.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
/tmp/ipykernel_7581/701967925.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(tolower)


In [124]:
offres_df_clean.head(5)

,Organisme de rattachement,Référence,Versant,Métier,Statut du poste,Nature de l'emploi,Durée du contrat,Intitulé du poste,Localisation du poste,Lieu d'affectation,Niveau d'études,Niveau d'expérience min. requis,Date de vacance de l'emploi,Date de début de publication par défaut,Date de fin de publication par défaut,Date de première publication,Spécialisation,Télétravail,Management,Employeur,Catégorie,Temps Plein,Durée du poste,Langues,Niveaux,Lieu d'affectation (sans géolocalisation),Avis de vacances au JO,Compétences attendues,Documents à transmettre,Nature de contrat
0,Structures de coopération territoriale,O043221200865282,Fonction Publique Territoriale,chargee charge de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,geomaticien ne ecologue syndicat mixte de gest...,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
1,Structures de coopération territoriale,O043221200864991,Fonction Publique Territoriale,chargee charge de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,charge e mission flore habitats naturels veget...,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
2,Structures de coopération territoriale,O043221200865428,Fonction Publique Territoriale,chargee charge de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,charge e de missions scientifiques et techniques,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
3,Structures de coopération territoriale,O064221200866576,Fonction Publique Territoriale,plombiere operatrice chauffagiste plombier ope...,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,agent de maintenance des batiments communaute ...,Pyrénées Atlantiques (64),Hôtel de la communauté de communes de lacq-ort...,NaN,NaN,01/03/2023,28/12/2022,27/01/2023,02/12/2022,NaN,NaN,NaN,NaN,Catégorie C (employé),Oui,NaN,NaN,NaN,Mourenx,NaN,NaN,NaN,NaN
4,Centres communaux d'Action Sociale,O032221000829833,Fonction Publique Territoriale,educatrice educateur de jeunes enfants,Vacant,Emploi ouvert uniquement aux contractuels,7 mois,puericulteur rice educateur rice de jeunes enf...,Gers (32),Mairie - 32500 FLEURANCE,NaN,NaN,09/01/2023,27/10/2022,26/12/2022,03/11/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Non,NaN,NaN,NaN,FLEURANCE,NaN,NaN,NaN,NaN


In [129]:
offres_df_clean[offres_df_clean['Métier'].str.contains("ux designer",na=False)|offres_df_clean['Intitulé du poste'].str.contains("data designer",na=False)][['Métier','Intitulé du poste']].head(5)

,Métier,Intitulé du poste
136654,architecte technique,expert conception et architecture de donnees d...
136668,architecte technique,expert conception et architecture de donnees d...
136672,architecte technique,expert conception et architecture de donnees d...


In [130]:
offres_df_clean[offres_df_clean['Métier'].str.contains("data scientist",na=False)|offres_df_clean['Intitulé du poste'].str.contains("data scientist",na=False)]['Organisme de rattachement'].unique()

array(['Centre Hospitalier Universitaire de Tours',
       "Muséum National d'Histoire Naturelle",
       'Direction générale des finances publiques (DGFiP)',
       "Institut national de l'information géographique et forestière",
       "Ministère de l'éducation nationale et de la jeunesse  -  Ministère de l'enseignement supérieur et de la recherche",
       'Secrétariat Général', 'Administration Centrale MENJ - MESR',
       'Autorité des Marchés Financiers',
       "Agence Régionale de Santé Provence-Alpes-Côte-d'Azur",
       'Agence de Services et de Paiement (ASP)',
       'Structures de coopération territoriale',
       'Direction Interministérielle du Numérique (DINUM)',
       'Agence Nationale de Sécurité du Médicament et des Produits de Santé',
       'Institut de Recherche pour le Développement',
       "Ministère de l'Intérieur",
       'Direction générale des entreprises (DGE)', 'DRSD',
       'Direction des Services Judiciaires',
       "Ministère des Armées - CMG d'Arcu

In [ ]:
offres_df_clean[]